# Learn Posture

use machine learning to recognize robot's posture (following the example in [scikit-learn-intro.ipynb](./scikit-learn-intro.ipynb) )

## 1. Data collection

We have colleceted data before, you need to add new data if you want to add new posture.

* the dateset are in *robot_pose_data* folder
* each file contains the data belongs to this posture, e.g. the data in *Back* file are collected when robot was in "Back" posture
* the data file can be load by ```pickle```, e.g. ```pickle.load(open('Back'))```, the data is a list of feature data
* the features (e.g. each row of the data) are ['LHipYawPitch', 'LHipRoll', 'LHipPitch', 'LKneePitch', 'RHipYawPitch', 'RHipRoll', 'RHipPitch', 'RKneePitch', 'AngleX', 'AngleY'], where 'AngleX' and 'AngleY' are body angle (e.g. ```Perception.imu```) and others are joint angles.

## 2. Data preprocessing

In [1]:
%pylab inline
import pickle
from os import listdir, path
import numpy as np
from sklearn import svm, metrics

ROBOT_POSE_DATA_DIR = 'robot_pose_data'

Populating the interactive namespace from numpy and matplotlib


In [2]:
classes = listdir(ROBOT_POSE_DATA_DIR)
print(classes)

['Back', 'Belly', 'Crouch', 'Frog', 'HeadBack', 'Knee', 'Left', 'Right', 'Sit', 'Stand', 'StandInit']


In [3]:
def load_pose_data(i):
    '''load pose data from file'''
    data = []
    target = []
    # YOUR CODE HERE
    
    filename = path.join(ROBOT_POSE_DATA_DIR, classes[i])
    data = pickle.load(open(filename))
    target = [i] * len(data)
    return data, target

In [11]:
# load all the data
all_data = []
all_target = []
# YOUR CODE HERE
for class_index in range(len(classes)):
    data, target = load_pose_data(class_index)
    all_data.extend(data)
    all_target.extend(target)

all_data = np.asarray(all_data)
all_target = np.asarray(all_target)
print('total number of data', len(all_data))

('total number of data', 222)


In [12]:
# shuffule data
permutation = np.random.permutation(len(all_data))
n_training_data = int(len(all_data) * 0.7)
training_data = permutation[:n_training_data]
test_data = permutation[n_training_data:]

## 3. Learn on training data

In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T). An example of an estimator is the class sklearn.svm.SVC that implements support vector classification.

In [13]:
clf = svm.SVC(gamma=0.001, C=100.)

### learning

In [14]:
# YOUR CODE HERE
clf.fit(all_data[training_data], all_target[training_data])

SVC(C=100.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

### predicting

In [15]:
clf.predict(all_data[training_data]), all_target[training_data]

(array([10, 10, 10,  9,  3,  3, 10, 10,  2,  2,  0,  8,  3,  4,  4,  1, 10,
         0, 10, 10,  7,  4, 10,  0, 10,  0,  0,  2,  9,  6, 10, 10,  5,  8,
         1,  5,  6,  2,  1,  2,  2,  2,  8, 10,  6,  8,  5,  2,  1,  8,  5,
         6,  2,  2,  6,  0,  6,  7,  9, 10,  9,  8,  8,  6,  1, 10, 10,  8,
         8,  3,  2, 10,  3, 10,  6,  3,  2,  8,  8,  2,  5,  1,  2, 10, 10,
         8,  8,  1,  3,  2, 10, 10,  8, 10,  1,  0,  7, 10,  0,  9,  9,  8,
        10,  5, 10,  4,  6,  1,  6,  1,  6,  5,  0, 10,  7,  1, 10,  7,  6,
         6,  1, 10,  2,  2,  2,  4,  2,  0, 10,  7,  9,  2, 10,  3,  1,  8,
        10,  0, 10,  5,  5,  2,  1,  0,  2,  2,  2,  1,  0,  5, 10,  6, 10,
        10,  8]),
 array([10, 10, 10,  9,  3,  3, 10, 10,  2,  2,  0,  8,  3,  4,  4,  1, 10,
         0, 10, 10,  7,  4, 10,  0, 10,  0,  0,  2,  9,  6, 10, 10,  5,  8,
         1,  5,  6,  2,  1,  2,  7,  2,  8, 10,  6,  8,  5,  2,  1,  8,  5,
         6,  2,  2,  6,  0,  6,  7,  9, 10,  9,  8,  8,  6,  1, 10, 10

In [16]:
def evaluate(expected, predicted):
    print("Classification report:\n%s\n" % metrics.classification_report(expected, predicted))

    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

In [17]:

expected = all_target[training_data]
predicted = clf.predict(all_data[training_data])

evaluate(expected, predicted)

Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       1.00      1.00      1.00        15
           2       0.96      1.00      0.98        23
           3       1.00      1.00      1.00         8
           4       1.00      1.00      1.00         5
           5       1.00      1.00      1.00        10
           6       1.00      1.00      1.00        14
           7       1.00      0.86      0.92         7
           8       1.00      1.00      1.00        17
           9       1.00      1.00      1.00         7
          10       1.00      1.00      1.00        36

   micro avg       0.99      0.99      0.99       155
   macro avg       1.00      0.99      0.99       155
weighted avg       0.99      0.99      0.99       155


Confusion matrix:
[[13  0  0  0  0  0  0  0  0  0  0]
 [ 0 15  0  0  0  0  0  0  0  0  0]
 [ 0  0 23  0  0  0  0  0  0  0  0]
 [ 0  0  0  8  0  0  0  0  0  0  0]
 

## 4. Evaluate on the test data

In [18]:

expected = all_target[test_data]
predicted = clf.predict(all_data[test_data])

evaluate(expected, predicted)

Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00         4
           2       1.00      1.00      1.00         7
           3       1.00      1.00      1.00         2
           4       1.00      1.00      1.00         5
           6       1.00      1.00      1.00         6
           7       1.00      1.00      1.00         4
           8       1.00      1.00      1.00         9
           9       1.00      1.00      1.00         4
          10       1.00      1.00      1.00        16

   micro avg       1.00      1.00      1.00        67
   macro avg       1.00      1.00      1.00        67
weighted avg       1.00      1.00      1.00        67


Confusion matrix:
[[10  0  0  0  0  0  0  0  0  0]
 [ 0  4  0  0  0  0  0  0  0  0]
 [ 0  0  7  0  0  0  0  0  0  0]
 [ 0  0  0  2  0  0  0  0  0  0]
 [ 0  0  0  0  5  0  0  0  0  0]
 [ 0  0  0  0  0  6  0  0  0  0]
 

## 5. Deploy to the real system

We can simple use `pickle` module to serialize the trained classifier.

In [21]:
import pickle
ROBOT_POSE_CLF = 'robot_pose.pkl'
pickle.dump(clf, open(ROBOT_POSE_CLF, 'wb'))

Then, in the application we can load the trained classifier again.

In [22]:
clf2 = pickle.load(open(ROBOT_POSE_CLF, 'rb'))
clf2.predict(all_data[-1].reshape(1, -1)), all_target[-1]

(array([10]), 10)